In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain_core.messages import HumanMessage, AIMessage
from langchain.tools.retriever import create_retriever_tool
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor

In [ ]:
loader = WebBaseLoader(["https://tomasellis.dev/about", "https://tomasellis.dev/projects"])
docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter()
embeddings = OpenAIEmbeddings()

In [ ]:
documents = text_splitter.split_documents(docs)

vector = FAISS.from_documents(documents, embeddings)

retriever = vector.as_retriever()

In [ ]:
retriever_tool = create_retriever_tool(
    retriever,
    "tomas_info",
    "Search for information about Tomas. For any questions about Tomas, you must use this tool!",
)

In [ ]:
tools = [retriever_tool]

In [ ]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
chat_history = [HumanMessage(content="Do you know Tomas?"), AIMessage(content="Yes!")]

In [ ]:

agent_executor.invoke({
    "chat_history": chat_history,
    "input": "What does Tomas do?"
})

In [106]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/agent/")
remote_chain.invoke({"input": "Can Tomas build bots?", "chat_history":[("human", "Do you know Tomas?"), ("ai", "Yes!")]})

HTTPStatusError: Server error '500 Internal Server Error' for url 'http://localhost:8000/agent/invoke'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500 for Internal Server Error